In [ ]:
from flask import Flask, request, render_template
import tensorflow as tf
import numpy as np
from PIL import Image

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict')
def predict():
    return render_template('upload.html')

# Load the trained models
covid_model = tf.keras.models.load_model('cov.h5')
pneumonia_model = tf.keras.models.load_model('pneumonia.h5')
tb_model = tf.keras.models.load_model('tb.h5')
cancer_model = tf.keras.models.load_model('cancer.h5')

# Define the labels
labels = ['COVID-19', 'Pneumonia', 'TB', 'Cancer']

@app.route('/detect', methods=['GET', 'POST'])
def detect():
    if request.method == 'POST':
        # Get the uploaded file from the request
        file = request.files['file']
        # Load and preprocess the input image
        img = Image.open(file).resize((224,224))
        img = np.array(img)
        if len(img.shape) == 2:
            img = np.stack((img,)*3, axis=-1)
        elif img.shape[2] == 1:
            img = np.repeat(img, 3, axis=2)
        img_array = img / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Make predictions using the trained models
        covid_prob = covid_model.predict(img_array)[0][0] * 100
        pneumonia_prob = pneumonia_model.predict(img_array)[0][0] * 100
        tb_prob = tb_model.predict(img_array)[0][0] * 100
        cancer_prob = cancer_model.predict(img_array)[0][0] * 100
        # Return the predicted probabilities to the HTML page
        return render_template('result.html', covid_prob=100-covid_prob, pneumonia_prob=pneumonia_prob, tb_prob=tb_prob, cancer_prob=100-cancer_prob)
    
    # If the request method is GET, return the HTML page to upload an image
    return render_template('home.html')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [29/Jun/2023 13:57:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2023 13:57:56] "GET /static/style1.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2023 13:57:56] "GET /static/v1.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [29/Jun/2023 13:57:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Jun/2023 13:58:12] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2023 14:02:40] "GET /static/v1.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [29/Jun/2023 14:02:40] "GET /static/v1.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [29/Jun/2023 14:02:41] "GET /static/v1.mp4 HTTP/1.1" 206 -
